# Sparse Coding of natural images and MNIST

Olshausen and Field, in their landmark paper in Nature, "Emergence of simple-cell receptive field properties by learning a sparse code for natural images", propose a method for representing images using a dictionary. 

One representation we have already worked with is PCA. In Assignment 2 Problem 3, we expressed each image as a combination of the top-$k$ principal vectors of the data. However, principal components are not localized. When we look at the principal components as images, we cannot visually isolate the feature they are representing. The Olshausen & Field sparse coding method attempts to solve this problem by generating a dictionary of localized "features" and express each image as a combination of as few of them as possible. 

Let $y^{(i)}\in\mathbb{R}^n$, for $i=1,\dots,N$ be the set of input images represented as $n$-dimensional vectors. We want to build a dictionary of size $p$, $X\in\mathbb{R}^{n\times d}$, where each column corresponds to an image representing a local feature. Let $\beta^{(i)}\in\mathbb{R}^d$ be the coefficient vector for image $i$. 

We optimize using the squared loss:
$$\begin{align}
\min_{\beta, X} & \;\; \sum_{i=1}^N \left\{\frac{1}{2n}\left\| y^{(i)} - X \beta^{(i)} \right\|_2^2 + \lambda \cdot \text{SparsityPenalty}(\beta^{(i)}) \right\}\\
\text{such that} & \;\; \| X_j\|_2 \leq 1
\end{align}$$

The features/basis vectors in the dictionary $X$ need not be orthogonal. Ideally, we would optimize both varibles $\beta$ and $X$ together, however the problem is not jointly convex. But for a fixed $X$, the problem is convex in $\beta$, and vice versa.

In this lab, we alternately update $X$ and $\beta$. In each iteration, we use stochastic gradient descent to update $X$, then fit $\beta$ while keeping $X$ fixed. Details of the algorithm are in the assignment writeup. In the following sections, we will discuss methods to fit the coefficients $\beta$ for fixed $X$.


====================================================================================================================
__Summary of Notation__

* $N$ data points


* each data point is in $\mathbb{R}^n$


* $d$ atoms (codewords) in the dictionary (codebook)


* overcomplete: $d \gg n$ i.e. more atoms than input dimension






# Lasso

This part discusses one way to update $\beta$ when $X$ is fixed.

* The SparsityPenalty function in the above optimization problem tries to enforce zero-values in the vectors $\beta^{(i)}$. 


* In literature, $0$-"norm" of a vector is defined as the number of nonzeros in it, which is the most SparsityPenalty function. 


* However, using the $0$-"norm" makes the problem very difficult to solve. (ASIDE: It's a NP-hard discrete problem). In general, continuous approximations/relaxations of it are used.


* Recall the (squared) $2$-norm regularizer: $$\|\beta\|^2_2 = \sum_{i=1}^d \beta_i^2$$

  It penalize large absolute values of $\beta_j$s, but the penalty becomes small rapidly as the non-zero values go closer to zero. That means, small components of $\beta$ don't really pay!


* In this sense, its better to use $1$-norm regularizer: $$\|\beta\|_1 = \sum_{i=1}^d \left| \beta_i \right |$$

  Both large and small components of $\beta$ have to pay in this case.
  
  *Hint: think about the gradients of both regularizers. If you use l2 regularization, every iteration you deduct $ \eta \cdot \beta$, so larger compoenents shrink more (pay more). If you use l1 regularization, you'll deduct $\eta \cdot sgn(\beta)$ -- all components pay the same.*
  
  
* Thus we have the so-called **Lasso** method: 

    \begin{align}
    \min_{\beta^{(i)} }& \;\; \frac{1}{2n}\left\| y^{(i)} - X \beta^{(i)} \right\|_2^2 + \lambda \cdot \|\beta^{(i)}\|_1 \\
    \end{align}

   This is for one data point $y^{(i)}$, you will need to do this for all data points in Assignment 3.


* $1$-norm regularization is used to promote sparsity at other places, too.

# Orthogonal Matching Pursuit (OMP)

* Matching pursuit is another algorithm to sparsely fit multidimensional vectors (in our case, $y^{(i)}$) using a dictionary ($X$) of vectors taken from the same space ($\mathbb{R}^n$).


* The objective is achieved using a greedy strategy. Basis vectors (columns of $X$) from the dictionary are chosen in the order of their correlation with the residual $r = y^{(i)} - X \beta^{(i)}$ step by step, until the approximation is good enough. 


* In other words, in each step, the residual $r$ (which is initially equal to the data vector) is updated by subtracting its projection onto one atom of the dictionary, with which it has the highest correlation. Next step, you select another atom from the rest. (Why only the rest?)

  (_You'll get one atom from the rest, but you don't necessarily need to worry this in the computation._)


* The word __orthogonal__ means that _after each step of the OMP algorithm, residual is orthogonal to all the selected columns_.

====================================================================================================

*__Algorithm: Orthogonal matching pursuit__*

Input : data vector $y$, dictionary $X$, number of atoms (basis vectors) you want to choose $k$

Output : indices of chosen atoms $S$, coefficient vector $\beta$

- Initialize $r = y$, $\beta = 0$
- Initialize empty set $S$
- for $i = 1,\ldots, k$
    - Add $j^* = \arg\max_{j} |r^\top X_j| / \| X_j \|_2$ to $S$
    - Set $\beta_{j^*} = r^\top X_{j^*}$ and $r = r - \beta_{j^*} X_{j^*}$. A better but slightly more expensive approach is to refit $y$ here.
- (Optional, refit $y$ if you didn't refit in the loop.)
- return : $S, \; \beta$

*(Refitting: solve least squares regression on selected basis $\|y- X_S \beta_S\|^2$, obtain new $r$ and $\beta$)*